In [9]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
from keras.preprocessing import sequence
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import regularizers

from keras.optimizers import RMSprop as rm
from keras.optimizers import Adagrad as ada
from keras.optimizers import SGD as sgd

with open('../data/general/matches_2017_2018_v1.json') as matches_json:
    matches = pd.read_json(matches_json)

Team ball possesion test arrays

In [13]:
id = 60561
x_train_np = np.load('../data/match_' + str(id) + '/x_data.npy')
y_train_np = np.load('../data/match_' + str(id) + '/y_team_data.npy')

x_train_np = np.expand_dims(x_train_np, axis=0)
y_train_np = np.expand_dims(y_train_np, axis=0)

id = 60562
x_valid_np = np.load('../data/match_' + str(id) + '/x_data.npy')
y_valid_np = np.load('../data/match_' + str(id) + '/y_team_data.npy')

x_valid_np = np.expand_dims(x_valid_np, axis=0)
y_valid_np = np.expand_dims(y_valid_np, axis=0)

id = 60563
x_test_np = np.load('../data/match_' + str(id) + '/x_data.npy')
y_test_np = np.load('../data/match_' + str(id) + '/y_team_data.npy')

x_test_np = np.expand_dims(x_test_np, axis=0)
y_test_np = np.expand_dims(y_test_np, axis=0)

Individual player ball possesion test arrays

In [56]:
id = 60561
x_train_np = np.load('../data/match_' + str(id) + '/x_data.npy')
y_train_np = np.load('../data/match_' + str(id) + '/y_player_data.npy')

x_train_np = np.expand_dims(x_train_np, axis=0)
y_train_np = np.expand_dims(y_train_np, axis=0)

id = 60562
x_valid_np = np.load('../data/match_' + str(id) + '/x_data.npy')
y_valid_np = np.load('../data/match_' + str(id) + '/y_player_data.npy')

x_valid_np = np.expand_dims(x_valid_np, axis=0)
y_valid_np = np.expand_dims(y_valid_np, axis=0)

All match (304) arrays (304, ,)

In [ ]:
x_data_all = np.ndarray(shape=(len(matches['id']), data_height, data_width))
y_team_data_all = np.ndarray(shape=(len(matches['id']), data_height, label_team_width))
y_player_data_all = np.ndarray(shape=(len(matches['id']), data_height, label_player_width))
y_pass_data_all = np.ndarray(shape=(len(matches['id']), data_height, label_pass_width))

idx = 0
for id in matches['id']:
    x_data_all[idx] = np.load('../data/match_' + str(id) + '/x_data.npy')
    y_team_data_all[idx] = np.load('../data/match_' + str(id) + '/y_team_data.npy')
    y_player_data_all[idx] = np.load('../data/match_' + str(id) + '/y_player_data.npy')
    y_pass_data_all[idx] = np.load('../data/match_' + str(id) + '/y_player_data.npy')
    idx +=1

In [8]:
max_review_length = 100

x_train_np = sequence.pad_sequences(x_train_np, maxlen=max_review_length)
x_valid_np = sequence.pad_sequences(x_valid_np, maxlen=max_review_length)

print(x_train_np.shape)
print(x_valid_np.shape)

(1, 100, 87)
(1, 100, 87)


In [37]:
batch_size = 1
epochs = 100
hidden_dim = 1000
timestep = 6000

input_dim = 87
output_dim = 3

#input_dim = 87+29
#output_dim = 29

model = Sequential()
model.add(LSTM(hidden_dim, 
               input_shape=(timestep, input_dim), 
               return_sequences=True,
               #return_state=True,
               kernel_regularizer=regularizers.l2(0.01)))
#model.add(BatchNormalization())
#model.add(Dropout(0.1))
model.add(Dense(output_dim, activation='softmax'))

#opt = rm()
#opt = ada()
opt = sgd(lr=0.005, momentum=0.9, decay=1e-5, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history = model.fit(x=x_train_np, y=y_train_np, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=None, \
                    validation_split=0.0, validation_data=(x_valid_np, y_valid_np), shuffle=False, class_weight=None, \
                    sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

# make predictions
score = model.evaluate(x_test_np, y_test_np, batch_size=batch_size)
print(model.metrics_names)
print(score)

Train on 1 samples, validate on 1 samples
Epoch 1/100


 - 19s - loss: 2.7496 - acc: 0.3210 - val_loss: 2.7611 - val_acc: 0.3197


Epoch 2/100


 - 17s - loss: 2.7336 - acc: 0.3590 - val_loss: 2.7485 - val_acc: 0.3453


Epoch 3/100


...

 - 18s - loss: 2.0556 - acc: 0.7667 - val_loss: 2.4276 - val_acc: 0.5248


Epoch 100/100


 - 16s - loss: 2.0518 - acc: 0.7665 - val_loss: 2.4262 - val_acc: 0.5253


1/1 [==============================] - 3s 3s/step


['loss', 'acc']
[2.465447425842285, 0.5049999952316284]


In [39]:
# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

#model.save('model/model.h5')
#model.save_weights('model/model_weight.h5')
#json_data = model.to_json()

#with open('model/json_data.txt', 'w') as outfile:
#    json.dump(json_data, outfile)

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])
